In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
hdfs_path = "hdfs://hdfs-nn:9000/projeto/demo/bronze/draft/1976_to_2015_Drafts.csv"

In [ ]:
# Create a DataFrame from JSON data (automatically infer schema and data types)
# There are other file formats you can read from (e.g., csv, orc, parquet)
# https://spark.apache.org/docs/2.2.0/sql-programming-guide.html#data-sources

# Read Sillicon valley episodes data
drafts = spark.read.csv(hdfs_path, header = True, inferSchema = True )

In [ ]:
drafts.printSchema()
drafts.show()
drafts.toPandas()

In [ ]:
from pyspark.sql.functions import when
from pyspark.sql.functions import lit
import math
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
from pyspark.sql.functions import ceil


drafts_rounds = drafts.withColumn("Pkround", \
                  when ((drafts.Pk < 30 ), lit("1"))\
                  .otherwise (  ceil( col("Pk") / 30))\
)

drafts_rounds.toPandas()

In [ ]:
#ver o numero de rondas que existe

drafts_rounds.createOrReplaceTempView("drafts_rounds")

drafts_1 = spark.sql(
    """
    SELECT COUNT(DISTINCT Pkround) FROM drafts_rounds;
    """
)

drafts_1.toPandas()

In [ ]:
#verificar se apartir 1988 houve mais de 2 rondas

drafts_rounds.createOrReplaceTempView("drafts_rounds")

drafts_2 = spark.sql(
    """
    SELECT * 
    FROM drafts_rounds
    WHERE  Pk > 60 and Draft_Yr > 1988  ;
    """
)

drafts_2.toPandas()

In [ ]:
#verificar se as rondas estão corretas

drafts_rounds.createOrReplaceTempView("drafts_rounds")

drafts_3 = spark.sql(
    """
    SELECT * 
    FROM drafts_rounds
    WHERE  Pk > 30  and Pkround < 0  ;
    """
)

drafts_3.toPandas()

In [ ]:
drafts_clean = drafts_rounds.drop(drafts_rounds.BPM)\
.drop(drafts_rounds.VORP)\
.drop(drafts_rounds.Executive)\
.drop(drafts_rounds.Tenure)\
.drop(drafts_rounds.Exec_ID)\
.drop(drafts_rounds.Exec_draft_exp)\
.drop(drafts_rounds.attend_college)\
.drop(drafts_rounds.first_year)\
.drop(drafts_rounds.second_year)\
.drop(drafts_rounds.third_year)\
.drop(drafts_rounds.fourth_year)\
.drop(drafts_rounds.fifth_year)\



drafts_clean.toPandas()

In [ ]:
draft_clean = drafts_clean.drop(drafts_clean._c0)
draft_clean.toPandas()

In [ ]:
draft_clean_1 = draft_clean.withColumnRenamed("Yrs", "Yrs_Played")
       
draft_clean_1.toPandas()

In [ ]:

from pyspark.sql.functions import floor
draft_clean_2 = draft_clean_1.withColumn("Pk_in_round",  floor( (( col("Pk") / 30 ) - (col("Pkround")-1)) * 30) )
                                        
                                        
                                        


draft_clean_2.toPandas()
   
                                           

In [ ]:
#verificar a pick na ronda (Pk_in_round)

draft_clean_2.createOrReplaceTempView("draft_clean_2")

drafts_verify = spark.sql(
    """
    SELECT * 
    FROM draft_clean_2
    WHERE Pkround = 2 and  Pk_in_round > 60 ;
    """
)

drafts_verify.toPandas()

In [ ]:
draft_clean_2.printSchema()


In [ ]:
draft_clean_3 = draft_clean_2.withColumnRenamed("All.Star", "All_Star")\
.withColumnRenamed("Minutes.Played", "Minutes_Played")\
.withColumnRenamed("Points.per.Game", "Points_per_Game")\
.withColumnRenamed("Assits.per.Game", "Assits_per_Game")\
.withColumnRenamed("Win.Share", "Win_Share")\
.withColumnRenamed("Minutes.per.Game", "Minutes_per_Game")\
.withColumnRenamed("TRB.per.game", "TRB_per_game")\


       
draft_clean_3.toPandas()

In [ ]:
draft_clean_3.show()

In [ ]:
# you can choose any location in HDFS, just be organized 
# Your data lake will grow with time and will become a swamp
spark.sql(
    """
    CREATE DATABASE NBA LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/silver/NBA.db/'
    """
)

In [ ]:
spark.sql(
    """
    CREATE EXTERNAL TABLE NBA.sl_Pk_drafts_table (
        Player STRING, All_NBA INT, All_Star INT, Pk INT, Team STRING,
        College STRING, Yrs_Played INT, Games INT, Minutes_Played INT, PTS INT, TRB INT, AST INT,
        FG_Percentage DOUBLE, TP_Percentage DOUBLE, FT_Percentage DOUBLE, Minutes_per_Game DOUBLE,
        Points_per_Game DOUBLE, TRB_per_game DOUBLE, Assits_per_Game DOUBLE, Win_Share DOUBLE,
        WS_per_game DOUBLE, Pkround STRING, Pk_in_round LONG
    )
    USING DELTA
    PARTITIONED BY (
    Draft_Yr INT
    )
    LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/silver/NBA.db/sl_Pk_drafts_table'
    """
)

In [ ]:
draft_clean_3 \
    .write \
    .mode("overwrite") \
    .partitionBy("Draft_Yr") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .save("hdfs://hdfs-nn:9000/projeto/demo/silver/NBA.db/sl_Pk_drafts_table")

In [ ]:
# create gold database
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS G_NBA LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/'
    """
)

In [ ]:
# create Under_Pk_per_year table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS G_NBA.Under_Pk
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE G_NBA.Under_Pk (
    Player STRING, All_NBA INT, All_Star INT, Pk INT, Team STRING,
        College STRING, Yrs_Played INT, Games INT, Minutes_Played INT, PTS INT, TRB INT, AST INT,
        FG_Percentage DOUBLE, TP_Percentage DOUBLE, FT_Percentage DOUBLE, Minutes_per_Game DOUBLE,
        Points_per_Game DOUBLE, TRB_per_game DOUBLE, Assits_per_Game DOUBLE, Win_Share DOUBLE,
        WS_per_game DOUBLE, Pkround STRING, Pk_in_round LONG, Draft_Yr INT
    )
    USING DELTA

    LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Under_Pk/'
    """
)

In [ ]:
# create Top_Pk_per_year table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS G_NBA.Top_Pk
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE G_NBA.Top_Pk (
    Player STRING, All_NBA INT, All_Star INT, Pk INT, Team STRING,
        College STRING, Yrs_Played INT, Games INT, Minutes_Played INT, PTS INT, TRB INT, AST INT,
        FG_Percentage DOUBLE, TP_Percentage DOUBLE, FT_Percentage DOUBLE, Minutes_per_Game DOUBLE,
        Points_per_Game DOUBLE, TRB_per_game DOUBLE, Assits_per_Game DOUBLE, Win_Share DOUBLE,
        WS_per_game DOUBLE, Pkround STRING, Pk_in_round LONG,  Draft_Yr INT
    )
    USING DELTA
    
    LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Top_Pk/'
    """
)

In [ ]:
# create Top_Pk_per_year table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS G_NBA.Top_Pk_null
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE G_NBA.Top_Pk_null (
        All_Star INT,  Draft_Yr INT
    )
    USING DELTA
    
    LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Top_Pk_null/'
    """
)

In [ ]:
# create Under_Pk_per_year table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS G_NBA.Under_Pk_null
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE G_NBA.Under_Pk_null (
        All_Star INT, Draft_Yr INT
    )
    USING DELTA

    LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Under_Pk_null/'
    """
)

In [ ]:
# create Top_Pk_per_year table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS G_NBA.Top_Pk_not_null
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE G_NBA.Top_Pk_not_null (
        All_Star INT,  Draft_Yr INT
    )
    USING DELTA
    
    LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Top_Pk_not_null/'
    """
)

In [ ]:
# create Under_Pk_per_year table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS G_NBA.Under_Pk_not_null
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE G_NBA.Under_Pk_not_null (
        All_Star INT, Draft_Yr INT
    )
    USING DELTA

    LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Under_Pk_not_null/'
    """
)

In [ ]:
# read data from the silver tables
Pk_df = spark.table("NBA.sl_Pk_drafts_table")

In [ ]:

Pk_df_1 = Pk_df.filter((Pk_df.Pk > 10) )
    

In [ ]:
Pk_df_1.show()

In [ ]:
Pk_df_2 = Pk_df.filter((Pk_df.Pk < 11) )

In [ ]:
Pk_df_3 = Pk_df.filter((Pk_df.Pk < 11) & (Pk_df.All_Star == 0))                     

In [65]:
from pyspark.sql.functions import count
Pk_df_4 = Pk_df_3.groupBy("Draft_Yr") \
    .agg(
    count(Pk_df_3.All_Star).alias("All_Star")
)

In [69]:
Pk_df_5 = Pk_df.filter((Pk_df.Pk > 10) & (Pk_df.All_Star == 0)) 

In [71]:
from pyspark.sql.functions import count
Pk_df_6 = Pk_df_5.groupBy("Draft_Yr") \
    .agg(
    count(Pk_df_5.All_Star).alias("All_Star")
)

In [72]:
Pk_df_7 = Pk_df.filter((Pk_df.Pk < 11) & (Pk_df.All_Star != 0)) 

In [73]:
from pyspark.sql.functions import count
Pk_df_8 = Pk_df_7.groupBy("Draft_Yr") \
    .agg(
    count(Pk_df_7.All_Star).alias("All_Star")
)

In [74]:
Pk_df_9 = Pk_df.filter((Pk_df.Pk > 10) & (Pk_df.All_Star != 0)) 

In [75]:
from pyspark.sql.functions import count
Pk_df_10 = Pk_df_9.groupBy("Draft_Yr") \
    .agg(
    count(Pk_df_9.All_Star).alias("All_Star")
)

In [ ]:
Pk_df_7.createOrReplaceTempView("Pk_df_7")

Pk_df_15 = spark.sql(
    """
    SELECT * 
    FROM Pk_df_7
    WHERE (Draft_Yr = 1990)  ;
    """
)

Pk_df_15.toPandas()

In [ ]:
Pk_df_2.show()

In [ ]:
# write to delta table
Pk_df_1 \
  .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Top_Pk")



In [ ]:
Pk_df_2 \
   .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Under_Pk")

In [67]:
Pk_df_4 \
   .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Under_Pk_null")

In [76]:
Pk_df_6 \
  .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Top_Pk_null")

In [77]:
Pk_df_8 \
   .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Under_Pk_not_null")

In [78]:
Pk_df_10 \
  .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Top_Pk_not_null")

In [ ]:
Pk_df_8.createOrReplaceTempView("Pk_df_8")

Pk_df_13 = spark.sql(
    """
    SELECT *
    FROM Pk_df_
    Where (Draft_Yr = 1990) 

    """
)

Pk_df_13.toPandas()

In [ ]:
spark.table("G_NBA.Under_Pk").show()


In [ ]:
Pk_df_1.show()

In [ ]:
spark.table("G_NBA.Top_Pk").show()

In [ ]:
spark.table("gold_NBA.Under_Pk").show()

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Top_Pk/`
""").show()

In [ ]:
spark.sql("""
DROP TABLE IF EXISTS G_NBA.Top_Pk_presto
""").show()

In [ ]:

spark.sql("""
CREATE EXTERNAL TABLE G_NBA.Top_Pk_presto (
   Player STRING, All_NBA INT, All_Star INT, Pk INT, Team STRING,
        College STRING, Yrs_Played INT, Games INT, Minutes_Played INT, PTS INT, TRB INT, AST INT,
        FG_Percentage DOUBLE, TP_Percentage DOUBLE, FT_Percentage DOUBLE, Minutes_per_Game DOUBLE,
        Points_per_Game DOUBLE, TRB_per_game DOUBLE, Assits_per_Game DOUBLE, Win_Share DOUBLE,
        WS_per_game DOUBLE, Pkround STRING, Pk_in_round LONG,  Draft_Yr INT
    )
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Top_Pk/_symlink_format_manifest/'
""").show()

In [89]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Top_Pk_null/`
""").show()

++
||
++
++



In [90]:
spark.sql("""
DROP TABLE IF EXISTS G_NBA.Top_Pk_null_presto
""").show()

++
||
++
++



22/01/14 14:23:13 WARN DataSource: All paths were ignored:
  hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Top_Pk_null/_symlink_format_manifest


In [91]:
spark.sql("""
CREATE EXTERNAL TABLE G_NBA.Top_Pk_null_presto (
   All_Star INT,  Draft_Yr INT
    )
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Top_Pk_null/_symlink_format_manifest/'
""").show()

++
||
++
++



In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Under_Pk/`
""").show()

In [ ]:
spark.sql("""
DROP TABLE IF EXISTS G_NBA.Under_Pk_presto
""").show()

In [ ]:


spark.sql("""
CREATE EXTERNAL TABLE G_NBA.Under_Pk_presto (
    Player STRING, All_NBA INT, All_Star INT, Pk INT, Team STRING,
        College STRING, Yrs_Played INT, Games INT, Minutes_Played INT, PTS INT, TRB INT, AST INT,
        FG_Percentage DOUBLE, TP_Percentage DOUBLE, FT_Percentage DOUBLE, Minutes_per_Game DOUBLE,
        Points_per_Game DOUBLE, TRB_per_game DOUBLE, Assits_per_Game DOUBLE, Win_Share DOUBLE,
        WS_per_game DOUBLE, Pkround STRING, Pk_in_round LONG, Draft_Yr INT
    )
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Under_Pk/_symlink_format_manifest/'
""").show()

In [92]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Under_Pk_null/`
""").show()

++
||
++
++



In [93]:
spark.sql("""
DROP TABLE IF EXISTS G_NBA.Under_Pk_null_presto
""").show()

++
||
++
++



In [94]:
spark.sql("""
CREATE EXTERNAL TABLE G_NBA.Under_Pk_null_presto (
    All_Star INT,  Draft_Yr INT
    )
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Under_Pk_null/_symlink_format_manifest/'
""").show()

++
||
++
++



In [95]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Under_Pk_not_null/`
""").show()

++
||
++
++



In [96]:
spark.sql("""
DROP TABLE IF EXISTS G_NBA.Under_Pk_null_not_presto
""").show()

++
||
++
++



22/01/14 14:23:38 WARN DataSource: All paths were ignored:
  hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Under_Pk_not_null/_symlink_format_manifest


In [97]:
spark.sql("""
CREATE EXTERNAL TABLE G_NBA.Under_Pk_null_not_presto (
    All_Star INT,  Draft_Yr INT
    )
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Under_Pk_not_null/_symlink_format_manifest/'
""").show()

++
||
++
++



In [98]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Top_Pk_not_null/`
""").show()

++
||
++
++



In [99]:
spark.sql("""
DROP TABLE IF EXISTS G_NBA.Top_Pk_null_not_presto
""").show()

++
||
++
++



In [100]:
spark.sql("""
CREATE EXTERNAL TABLE G_NBA.Top_Pk_null_not_presto (
   All_Star INT,  Draft_Yr INT
    )
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/demo/gold/G_NBA.db/Top_Pk_not_null/_symlink_format_manifest/'
""").show()

++
||
++
++



In [ ]:
spark.table("G_NBA.Top_Pk_per_year_presto").show()

In [ ]:
spark.table("G_NBA.Under_Pk_per_year_presto").show()